In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import logging

import gradio as gr

#regression
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, HuberRegressor, Lasso
from sklearn.metrics import make_scorer, mean_squared_error,confusion_matrix,accuracy_score
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR  
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV 

#Classification
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [2]:
#logger
logging.basicConfig(filename='logs.log', level=logging.INFO,format='%(asctime)s :: %(levelname)s :: %(message)s')

### Data Exploration

In [3]:
#reading and labeling data
def DataReading(filename):
    data=pd.read_csv(filename, sep=" ", header=None)
    # columns list
    COLS = ["ID", "Cycle", "OpSet1", "OpSet2", "OpSet3", "SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]
    data =data[[f for f in range(0, 26)]]
    data.columns = ["ID", "Cycle", "OpSet1", "OpSet2", "OpSet3", "SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]
    return data
    



#train data
train_data=DataReading("data set/aero/train_FD001.txt")

#test data
test_data=DataReading("data set/aero/test_FD001.txt")

logging.info("Training and Test Data Collection and labeling complete")


In [4]:
# reading Rul of last cycle of each engin for test data
#reading data train data

Rul_data=pd.read_csv("data set/aero/RUL_FD001.txt", sep=" ", header=None)
Rul_data["ID"] = Rul_data.index + 1
Rul_data=Rul_data.rename(columns={0:"RUL"}).drop(1,axis=1)
Rul_data
logging.info("Remaining Usefull life of Test Data set reading/Loading complete")

THIS THE RUL OF LAST CYCLE OF DIFFRENT ENGINS OF TEST DATA

### Preprocessing

In [5]:
# maximun number of engines cycle in train data
max_cycles_df_train = train_data.groupby(["ID"], sort=False)["Cycle"].max().reset_index().rename(columns={"Cycle" : "MaxCycleID"})
max_cycles_df_train



# Calculating RUL form the max_cycles_df in train data
train_data = pd.merge(train_data, max_cycles_df_train, how="inner", on="ID")
train_data["RUL"] = train_data["MaxCycleID"] - train_data["Cycle"]
train_data.drop('MaxCycleID',inplace=True, axis=1)
train_data.head()

#coverting into binary Rul 
# 1 for okay
# 0 for not Okay
binary_RUL=train_data['RUL'].map(lambda x: 1 if x>=20 else 0)
train_data['Binary_RUL']=binary_RUL
pro_train_data=train_data
pro_train_data
logging.info("Training Data Set RUL Calculation and Binary coversion of RUL complete")

In [6]:
# maximun number of engines cycle in test data
max_cycles_df_test = test_data.copy(deep=True).groupby(['ID'])["Cycle"].max().reset_index()
max_cycles_df_test

test_data = pd.merge(test_data, max_cycles_df_test, how="inner", on=["ID",'Cycle'])
test_data["RUL"]=Rul_data["RUL"]

#coverting into binary Rul 
# 1 for okay
# 0 for not Okay
binary_RUL=test_data['RUL'].map(lambda x: 1 if x>=20 else 0)
test_data['Binary_RUL']=binary_RUL
pro_test_data=test_data
logging.info("Testing Data Set Binary coversion of RUL complete")
pro_test_data

,ID,Cycle,OpSet1,OpSet2,OpSet3,SensorMeasure1,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure5,...,SensorMeasure14,SensorMeasure15,SensorMeasure16,SensorMeasure17,SensorMeasure18,SensorMeasure19,SensorMeasure20,SensorMeasure21,RUL,Binary_RUL
0,1,31,-0.0006,0.0004,100.0,518.67,642.58,1581.22,1398.91,14.62,...,8130.11,8.4024,0.03,393,2388,100.0,38.81,23.3552,112,1
1,2,49,0.0018,-0.0001,100.0,518.67,642.55,1586.59,1410.83,14.62,...,8126.90,8.4505,0.03,391,2388,100.0,38.81,23.2618,98,1
2,3,126,-0.0016,0.0004,100.0,518.67,642.88,1589.75,1418.89,14.62,...,8131.46,8.4119,0.03,395,2388,100.0,38.93,23.2740,69,1
3,4,106,0.0012,0.0004,100.0,518.67,642.78,1594.53,1406.88,14.62,...,8133.64,8.4634,0.03,395,2388,100.0,38.58,23.2581,82,1
4,5,98,-0.0013,-0.0004,100.0,518.67,642.27,1589.94,1419.36,14.62,...,8125.74,8.4362,0.03,394,2388,100.0,38.75,23.4117,91,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,97,-0.0006,0.0003,100.0,518.67,642.30,1590.88,1397.94,14.62,...,8148.24,8.4110,0.03,391,2388,100.0,38.96,23.4606,137,1
96,97,134,0.0013,-0.0001,100.0,518.67,642.59,1582.96,1410.92,14.62,...,8155.48,8.4500,0.03,395,2388,100.0,38.61,23.2953,82,1
97,98,121,0.0017,0.0001,100.0,518.67,642.68,1599.51,1415.47,14.62,...,8146.39,8.4235,0.03,394,2388,100.0,38.76,23.3608,59,1
98,99,97,0.0047,-0.0000,100.0,518.67,642.00,1585.03,1397.98,14.62,...,8150.38,8.4003,0.03,391,2388,100.0,38.95,23.3595,117,1


In [7]:
pd.set_option('display.max_columns', None)
pro_test_data

,ID,Cycle,OpSet1,OpSet2,OpSet3,SensorMeasure1,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure5,SensorMeasure6,SensorMeasure7,SensorMeasure8,SensorMeasure9,SensorMeasure10,SensorMeasure11,SensorMeasure12,SensorMeasure13,SensorMeasure14,SensorMeasure15,SensorMeasure16,SensorMeasure17,SensorMeasure18,SensorMeasure19,SensorMeasure20,SensorMeasure21,RUL,Binary_RUL
0,1,31,-0.0006,0.0004,100.0,518.67,642.58,1581.22,1398.91,14.62,21.61,554.42,2388.08,9056.40,1.3,47.23,521.79,2388.06,8130.11,8.4024,0.03,393,2388,100.0,38.81,23.3552,112,1
1,2,49,0.0018,-0.0001,100.0,518.67,642.55,1586.59,1410.83,14.62,21.61,553.52,2388.10,9044.77,1.3,47.67,521.74,2388.09,8126.90,8.4505,0.03,391,2388,100.0,38.81,23.2618,98,1
2,3,126,-0.0016,0.0004,100.0,518.67,642.88,1589.75,1418.89,14.62,21.61,552.59,2388.16,9049.26,1.3,47.88,520.83,2388.14,8131.46,8.4119,0.03,395,2388,100.0,38.93,23.2740,69,1
3,4,106,0.0012,0.0004,100.0,518.67,642.78,1594.53,1406.88,14.62,21.61,552.64,2388.13,9051.30,1.3,47.65,521.88,2388.11,8133.64,8.4634,0.03,395,2388,100.0,38.58,23.2581,82,1
4,5,98,-0.0013,-0.0004,100.0,518.67,642.27,1589.94,1419.36,14.62,21.61,553.29,2388.10,9053.99,1.3,47.46,521.00,2388.15,8125.74,8.4362,0.03,394,2388,100.0,38.75,23.4117,91,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,97,-0.0006,0.0003,100.0,518.67,642.30,1590.88,1397.94,14.62,21.61,553.99,2388.03,9062.41,1.3,47.14,522.30,2388.01,8148.24,8.4110,0.03,391,2388,100.0,38.96,23.4606,137,1
96,97,134,0.0013,-0.0001,100.0,518.67,642.59,1582.96,1410.92,14.62,21.61,554.05,2388.06,9076.36,1.3,47.38,521.58,2388.06,8155.48,8.4500,0.03,395,2388,100.0,38.61,23.2953,82,1
97,98,121,0.0017,0.0001,100.0,518.67,642.68,1599.51,1415.47,14.62,21.61,553.44,2388.13,9062.34,1.3,47.66,521.53,2388.09,8146.39,8.4235,0.03,394,2388,100.0,38.76,23.3608,59,1
98,99,97,0.0047,-0.0000,100.0,518.67,642.00,1585.03,1397.98,14.62,21.61,554.75,2388.01,9067.16,1.3,47.26,521.82,2388.02,8150.38,8.4003,0.03,391,2388,100.0,38.95,23.3595,117,1


From the above graphs we can see that opset3 ,SensorMeasure 1,5,6,10,16,18,19 are constant for diffrent RULs.
so these features will not be use full for the training.